In [1]:
import sys
import argparse
import os
import glob

In [2]:
# Script to aligned ORF against orthologous regions. Requirements: BLAST
# Arguments
# parser = argparse.ArgumentParser(description="Arguments for this script")
# parser.add_argument("--prot_dir", type=str, help="(Required) Directory including fasta with all protein sequence of ortholog regions (output of 1_extract_multiple_alignments.py)")
# parser.add_argument("--prot_tar", type=str, help="(Required) Fasta including all translated sequences encoded by the ORFs")
# args = parser.parse_args()

# Main
# assign each species to the lineage
lineages = ["human","hominoid","catarrhini","simiiformes","primates","primatomorpha","euarchontoglires","boreoeutheria","placentalia","mammal"]
order = {"hg38":"human"}
for sp in ("panTro5","panPan2","gorGor5","ponAbe2","nomLeu3"):
	order[sp] = "hominoid"
for sp in ("rheMac8","macFas5","macNem1","papAnu3","manLeu1","cerAty1","chlSab2","nasLar1","rhiRox1","rhiBie1","colAng1","HLpilTep1"):
	order[sp] = "catarrhini"
for sp in ("calJac3","aotNan1","saiBol1","cebCap1"):
	order[sp] = "simiiformes"
for sp in ("tarSyr2","otoGar3","micMur3","proCoq1"):
	order[sp] = "primates"
for sp in ("galVar1",):
	order[sp] = "primatomorpha"	
for sp in ("tupChi1","jacJac1","micOch1","criGri1","mesAur1","perManBai1","mm10","HLmusCar1","HLmusPah1","rn6","HLmerUng1","nanGal1","HLcasCan1","HLdipOrd2","hetGla2","HLfukDam1","cavPor3","chiLan1","octDeg1","speTri2","HLmarMar1","oryCun2","ochPri3"): #rodents+primates
	order[sp] = "euarchontoglires"

cetartiodactyla =("vicPac2","HLcamFer2","HLcamBac1","HLcamDro1","HLturTru3","HLorcOrc1","HLdelLeu1","lipVex1","phyCat1","balAcu1","HLbalMys1","bosTau8","HLbosInd1","bisBis1","bosMut1","bubBub1","HLoviAri4","HLoviCan1","HLcapHir2","panHod1","HLodoVir1","HLcerEla1","susScr11")
perissodactyla = ("HLequCab3","equPrz1","HLequAsi1","cerSim1")
ferae = ("felCat8","HLaciJub1","panTig1","HLpanPar1","canFam3","HLlycPic1","musFur1","HLenhLut1","HLailFul1","ailMel1","ursMar1","odoRosDiv1","lepWed1","neoSch1","manPen1","HLmanJav1")
chiroptera = ("pteAle1","HLpteVam2","rouAeg1","HLrhiSin1","HLhipArm1","eptFus1","myoDav1","myoBra1","myoLuc2","HLminNat1","HLdesRot1")
eulipotyphla = ("eriEur2","sorAra2","conCri1")
boreoeutheria = cetartiodactyla + perissodactyla + ferae + chiroptera + eulipotyphla

for sp in boreoeutheria: #previous+euarchontoglires
	order[sp] = "boreoeutheria"
for sp in ("loxAfr3","triMan1","HLproCap2","chrAsi1","echTel2","eleEdw1","oryAfe1","dasNov3","HLchoHof2"): #atlantogenata +boro
	order[sp] = "placentalia"
for sp in ("monDom5","sarHar1","HLphaCin1","ornAna2"): #nonplacental+rest
	order[sp] = "mammal"

In [3]:
len(order)

120

In [4]:
maf_path="/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/results_120/orfs/ENST00000514422.1+chr4__3589823-3589865.maf"

In [5]:
from Bio import SeqIO
from Bio.Seq import Seq
fan = {}
maf = SeqIO.index(maf_path, "fasta")
for n in maf:
	fan[n] = str(Seq(str(maf[n].seq).replace("-",""))).upper()

In [6]:
fan['hg38'][0:3]

'ATG'

In [7]:
fan

{'hg38': 'ATGCCGTCTATTCTTGACAGAAGTGCTTTATGTGGCACA',
 'panTro5': 'ATGCCGTCTGTTCTTGACGGAAGTGCTGTTACGTGGCACA',
 'gorGor5': 'ATGCCGTCTGTTCTTGACGGAAGTGCTGTTATGTGGCACA',
 'ponAbe2': 'ATGCGGTCCATTCTTGACAGACGTGCTGTTATGTGGCACA',
 'nomLeu3': 'ATGCCGTCTGTTCTTGACAGAAGTGCTGTTATATGGCACA',
 'rheMac8': 'GTGCTCTCTTTTCCTGACAGAAGTGCTGTTATGTGGCACA',
 'macFas5': 'GTGCTCTCTTTTCCTGACAGAAGTGCTGTTATGTGGCACA',
 'macNem1': 'GTGCTCTCTTTTCCTGACAGAAGTGCTGTTATGTGGCACA',
 'papAnu3': 'GTGCTCTCTTTTCCTGACAGAAGTGTTGTTATGTGGCACA',
 'manLeu1': 'ATGCTCTCTGTTCCTGAGAGAAGTGTTGTTATGTGGCACA',
 'cerAty1': 'ATGCTCTCTGTTCCTGAGAGAAGTGCTGTTATGTGGCACA',
 'chlSab2': 'ATGCTCTCTGTTCCTGAGAGAAGTGCTGTTATGTGGCACA',
 'rhiRox1': 'ATGCTCTCTGTTCCTGCCAGAAGTGCTGTTATGTGGCACA',
 'rhiBie1': 'ATGCTCTCTGTTCCTGCCAGAAGTGCTGTTATGTGGCACA',
 'colAng1': 'ATGCTCTCTGTTCCTGCCAGAATTGCTGTTATGTGGCACA',
 'HLpilTep1': 'ATGCTCTCTGTTCCTGCCAGAAGTGCTGTTATGTGGCACA',
 'calJac3': 'ATGCAGTCCGTTCTCGACAGAAGTGGCACA',
 'aotNan1': 'GTGCGGTCCATTCTTGACAGAAGTGGCACG',
 'saiBol1': 'A

In [8]:
# Generate all truncated protein if start is a ATG
prot_ort = "/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/results_120/orfs/ENST00000514422.1+chr4__3589823-3589865.fa"
al_scodon = ["ATG", "CTG", "ACG", "GTG", "TTG", "ATA", "ATC", "ATT", "AAG", "AGG"]
for file in glob.glob(prot_ort):
    s = ""
    species = "hg38"
    trunc_file = file.replace(".fa", ".trunc.fa")
    if os.path.exists(trunc_file):
        os.remove(trunc_file)
    outs = open(file.replace(".fa",".trunc.fa"),"w+")
    for line in open(file):
        tru_seq = ""
        new = -1
        if ">" in line:
            s = line.split()[0].replace(">","")
            species = s.split("_")[2]
            first_scodon=fan[species][0:3]
            second_codon=fan[species][3:6]
            third_codon=fan[species][6:9]
        else:
            for n,c in enumerate(str(line).rstrip("\n")):
                # 如果第1-3个密码子的位置为可能的起始密码子中的一个，初始值new=-1
                if (first_scodon in al_scodon or second_codon in al_scodon or third_codon in al_scodon) and (new == -1): # in-frame window 6 nt downstream
                    outs.write(">" + s + "_" + str(n) + "\n")
                    new = 1
                    tru_seq += c
                # if stop
                # 继续读的过程中遇到了终止密码子
                elif c == "*" and (new == 1):
                    tru_seq += c
                    new = -1
                    outs.write(tru_seq + "\n")
                    tru_seq = ""
                    break
                # if end of ortholog region
                # 如果读到了末尾还没有终止密码子
                elif n == len(str(line).rstrip("\n"))-1 and (new == 1):
                    tru_seq += c
                    new = -1
                    outs.write(tru_seq + "\n")
                elif new == 1:
                    tru_seq += c
    outs.close()

In [9]:
# Perform blastp ORF peptide against the ortholog peptide sequences
prot_ort = "/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/results_120/orfs/ENST00000514422.1+chr4__3589823-3589865.trunc.fa"
prot_tar = "/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/peptide_fa/ENST00000514422.1+chr4:3589823-3589865.ORF_pep.fa"
out = open(prot_ort.replace(".trunc.fa",".all.prot_spec.out"),"w+")

In [10]:
# 默认的e value的阈值是10
os.system("/home/user/data3/rbase/opt/ncbi-blast-2.14.0+/bin/blastp -query " + prot_tar + " -subject " + prot_ort +
            " -subject_besthit -outfmt \"6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen qcovs evalue bitscore\" >  " + 
            prot_ort.replace(".fa",".blastp.out"))

0

In [11]:
prot_tar
# prot_ort

'/home/user/data3/lit/project/sORFs/05-denovo-status/analysis/in_house_human_brain_denovo_check_20250410/peptide_fa/ENST00000514422.1+chr4:3589823-3589865.ORF_pep.fa'

In [12]:
out.write("orf_id\tlineage\tfarthest species\tpident\tevalue\tqcovs\n")
for file in glob.glob(prot_ort):
    blastp = {}
    os.system("/home/user/data3/rbase/opt/ncbi-blast-2.14.0+/bin/blastp -query " + prot_tar + " -subject " + file +
            " -subject_besthit -outfmt \"6 qseqid sseqid pident length mismatch gapopen qstart qend qlen sstart send slen qcovs evalue bitscore\" >  " + 
            file.replace(".fa",".blastp.out"))
    # read blastp results
    for line in open(file.replace(".fa",".blastp.out")):
        # other species
        sseqid = line.split("\t")[1]
        if not sseqid in blastp:
            blastp[sseqid] = []
        pident = float(line.split("\t")[2])
        evalue = float(line.split("\t")[13])
        qcovs = float(line.split("\t")[12])
        blastp[sseqid].append(pident)
        blastp[sseqid].append(evalue)
        blastp[sseqid].append(qcovs)

    # Find farthest species and lineage
    far_sp = ""
    far_idx = 0
    for k,v in blastp.items():
        # 需要identity大于等于80且相似度大于70
        if (v[0] >= 80) and (v[2] >= 70):
            print("{0}: {1}".format(k,v))
            species = k.split("_")[2]
            i = lineages.index(order[species])
            if i >= far_idx:
                far_sp = k
                far_idx = i

    print("This protein is " + lineages[far_idx] + "-specific.")
    orf_id = far_sp.split("_")[0]
    pident = blastp[far_sp][0]
    evalue = blastp[far_sp][1]
    qcovs = blastp[far_sp][2]
    out.write(orf_id + "\t" + lineages[far_idx] + "\t" + str(far_sp) + "\t" + str(pident) + "\t" + str(evalue) + "\t" + str(qcovs) + "\n")

out.close()

ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_hg38_1.0_0: [100.0, 4.56e-08, 100.0]
ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_nomLeu3_1.0_0: [80.0, 7.41e-05, 77.0]
This protein is hominoid-specific.


In [13]:
blastp

{'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_hg38_1.0_0': [100.0,
  4.56e-08,
  100.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_nomLeu3_1.0_0': [80.0,
  7.41e-05,
  77.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_panTro5_1.0_0': [70.0,
  0.001,
  77.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_gorGor5_1.0_0': [70.0,
  0.001,
  77.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_calJac3_1.0_0': [66.667,
  0.004,
  69.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_cebCap1_1.0_0': [77.778,
  0.004,
  69.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_saiBol1_1.0_0': [77.778,
  0.004,
  69.0],
 'ENST00000514422.1+chr4:3589823-3589865_ENST00000514422.1+chr4:3589823-3589865_aotNan1_1.0_0': [85.714,
  0.022,
  54.0]}